# Data Warehouse Truven QA - Member Enrollment Yearly

Performing QA on tables in dw_staging before moving them to data_warehouse schema

## Initialization

Just loading packages that will be used and initializing connection to GP DB.

In [2]:
import pandas as pd
import sys
import psycopg2
from tqdm.notebook import tqdm
sys.path.append('H:/uth_helpers')
from db_utils import get_dsn

In [3]:
connection = psycopg2.connect(get_dsn())
connection.autocommit = True

In [3]:
connection.close()

## Member Counts and Member Months

This table is an aggregate of the member_enrollment_monthly table where enrollment data is aggregated to a yearly level.

Because of this, we cannot compare the row count of the raw tables with this table since enrollment tables from raw data sources are usually not on a yearly level. Instead, we can try to compare the Member Months (MM) and other columns to see if the aggregrate logic implemented is working as intended

In [5]:
query = '''drop table if exists qa_reporting.dw_truv_mbr_enrl_yearly;
create table qa_reporting.dw_truv_mbr_enrl_yearly
(
    data_source text,
    calendar_year int,
    table_src text,
    dw_row_count int,
    dw_uth_mbr_id_count int,
    dw_src_mbr_id_count int,
    src_mbr_count int,
    mbr_count_diff int,
    mbr_count_percentage float,
    dw_mm int,
    src_mm int,
    mm_diff int,
    mm_diff_percentage float,
    date_generated date
);
'''

with connection.cursor() as cursor:
    cursor.execute(query)

In [6]:
with connection.cursor() as cursor:
      query = '''
insert into qa_reporting.dw_truv_mbr_enrl_yearly
(data_source, calendar_year, table_src, dw_row_count, dw_uth_mbr_id_count, dw_src_mbr_id_count, dw_mm,date_generated)
select data_source, 
        year, 
        table_id_src, 
        count(*),
        count(distinct uth_member_id),
        count(distinct member_id_src),
        sum(total_enrolled_months),
        now()::date
  from dw_staging.truv_member_enrollment_yearly
 group by 1,2,3;
      '''

      cursor.execute(query)


In [8]:
with connection.cursor() as cursor:
    
      query = '''
update qa_reporting.dw_truv_mbr_enrl_yearly a
set src_mbr_count = b.pat_count,
    mbr_count_diff = a.dw_src_mbr_id_count - b.pat_count,
    mbr_count_percentage = abs(a.dw_src_mbr_id_count - b.pat_count) / b.pat_count
from qa_reporting.truven_counts b
where data_source = 'truv'
and a.calendar_year = b.year
and a.table_src = b.table_name
;
      '''

      cursor.execute(query)

      query = '''
with enrl_months as(
select 'ccaet' as table_name, year, enrolid, extract(month from dtstart) as enrolled_month
from truven.ccaet
union
select 'mdcrt' as table_name, year, enrolid, extract(month from dtstart) as enrolled_month
from truven.mdcrt
),
enrl_my as (
    select table_name, year, count(enrolled_month) as mm
    from enrl_months
    group by 1,2
)
update qa_reporting.dw_truv_mbr_enrl_yearly a
set src_mm = b.mm,
    mm_diff = a.dw_mm - b.mm,
    mm_diff_percentage = abs(a.dw_mm - b.mm) / b.mm
from enrl_my b
where data_source = 'truv'
and a.calendar_year = b.year
and a.table_src = b.table_name
;
      '''

      cursor.execute(query)

In [9]:
df = pd.read_sql('''select * 
from qa_reporting.dw_truv_mbr_enrl_yearly 
order by mm_diff ;''', con=connection)
df

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,data_source,calendar_year,table_src,dw_row_count,dw_uth_mbr_id_count,dw_src_mbr_id_count,src_mbr_count,mbr_count_diff,mbr_count_percentage,dw_mm,src_mm,mm_diff,mm_diff_percentage,date_generated
0,truv,2012,mdcrt,4673161,4673161,4673161,4874717,-201556,0.0,51027886,51474029,-446143,0.0,2023-04-18
1,truv,2013,mdcrt,4093681,4093681,4093681,4271755,-178074,0.0,44817186,45238676,-421490,0.0,2023-04-18
2,truv,2014,mdcrt,3712364,3712364,3712364,3868830,-156466,0.0,40881107,41158746,-277639,0.0,2023-04-18
3,truv,2011,mdcrt,5049811,5049811,5049811,5243029,-193218,0.0,56450926,56639459,-188533,0.0,2023-04-18
4,truv,2019,mdcrt,1588597,1588597,1588597,1632440,-43843,0.0,18185358,18237778,-52420,0.0,2023-04-18
5,truv,2021,mdcrt,1351234,1351234,1351234,1387227,-35993,0.0,15401613,15439488,-37875,0.0,2023-04-18
6,truv,2020,mdcrt,1659504,1659504,1659504,1703033,-43529,0.0,18958930,18995409,-36479,0.0,2023-04-18
7,truv,2018,ccaet,27050874,27050874,27050874,27087740,-36866,0.0,275916853,275947642,-30789,0.0,2023-04-18
8,truv,2022,mdcrt,1326303,1326303,1326303,1342635,-16332,0.0,7757558,7775757,-18199,0.0,2023-04-18
9,truv,2015,ccaet,28284784,28284784,28284784,28348363,-63579,0.0,289792102,289793751,-1649,0.0,2023-04-18


Here we look at the overall difference of member enrollment and member count. We start of by just adding the counts for each of the source tables. However, due to how the yearly table is built, it may be better to look at member counts and member months as the data source overall.

In [11]:
df.groupby('calendar_year')[['mm_diff', 'mm_diff_percentage', 'mbr_count_diff', 'mbr_count_percentage']].sum()

,mm_diff,mm_diff_percentage,mbr_count_diff,mbr_count_percentage
calendar_year,,,,
2011,-3,0.0,-340755,0.0
2012,-4,0.0,-334300,0.0
2013,-2,0.0,-282938,0.0
2014,0,0.0,-262178,0.0
2015,0,0.0,-129457,0.0
2016,0,0.0,-130816,0.0
2017,-1,0.0,-89452,0.0
2018,0,0.0,-69922,0.0
2019,0,0.0,-78353,0.0


Now we will look at the overall member count and member months rather than splitting them up based on the source table.

In [12]:
query = '''
with truv_enrl as (
    -- get unique members and their monthly enrollment to any program per calendar year
    select year, enrolid, extract(month from dtstart) as enrolled_month
    from truven.ccaet
    union
    select year, enrolid, extract(month from dtstart) as enrolled_month
    from truven.mdcrt
),
truv_enrl_counts as (
    select year, count(distinct enrolid) src_mbr_count, count(enrolled_month) src_mm
    from truv_enrl
    group by 1
),
dw_truv_enrl as (
    select year, count(uth_member_id) as dw_mbr_count, sum(total_enrolled_months) as dw_mm
    from dw_staging.truv_member_enrollment_yearly
    group by 1
)
select a.year, a.dw_mbr_count, b.src_mbr_count, 
        a.dw_mbr_count - b.src_mbr_count as mbr_diff,
        a.dw_mm, b.src_mm, a.dw_mm - b.src_mm as mm_diff
from dw_truv_enrl a
join truv_enrl_counts b
on a.year = b.year;'''

pd.read_sql(query, con=connection)

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,year,dw_mbr_count,src_mbr_count,mbr_diff,dw_mm,src_mm,mm_diff
0,2016,30725641,30725641,0,320175484,320175484,0
1,2019,26942865,26942865,0,276086832,276086832,0
2,2020,24929462,24929462,0,260249638,260249638,0
3,2014,50865180,50865180,0,516344877,516344877,0
4,2017,27530610,27530610,0,287386858,287386858,0
5,2022,20831493,20831493,0,114985773,114985773,0
6,2013,47726034,47726034,0,487691304,487691304,0
7,2011,60461428,60461428,0,621327259,621327259,0
8,2015,30418539,30418539,0,314040663,314040663,0
9,2018,28148431,28148431,0,288516600,288516600,0


## Gender Counts

Just like with the monthly enrollment table, we need to check the counts and values of the other columns. Here we take a look at the gender_cd column

In [14]:
query = '''with truven_gen_cd as (
    select year, enrolid, sex
    from truven.ccaet
    union
    select year, enrolid, sex
    from truven.mdcrt
),
truven_gen as (
    select year, c.gender_cd, count(*) gender_count
    from truven_gen_cd m
    left outer join reference_tables.ref_gender c
    on c.data_source = 'trv'
    and c.gender_cd_src = m.sex::text
    group by 1,2
), dw_gen as (
    select year, gender_cd, count(*) gender_count
    from dw_staging.truv_member_enrollment_yearly
    group by 1,2
)
select a.*, b.gender_count as src_gender_count, 
        a.gender_count - b.gender_count as gender_diff, 
        100. * abs(a.gender_count - b.gender_count) / b.gender_count as gender_diff_percent
from truven_gen b
full outer join dw_gen a
on a.year = b.year
and a.gender_cd = b.gender_cd
order by year, gender_cd;
'''

pd.read_sql(query,  con=connection)

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,year,gender_cd,gender_count,src_gender_count,gender_diff,gender_diff_percent
0,2011,F,31157196,31157196,0,0.0
1,2011,M,29304232,29304232,0,0.0
2,2012,F,31152383,31152383,0,0.0
3,2012,M,29363662,29363662,0,0.0
4,2013,F,24607955,24607955,0,0.0
5,2013,M,23118079,23118079,0,0.0
6,2014,F,26186789,26186789,0,0.0
7,2014,M,24678391,24678391,0,0.0
8,2015,F,15814141,15814141,0,0.0
9,2015,M,14604398,14604398,0,0.0


## Plan Type

Check if plan types are properly mapped at a yearly level. If the plan type counts do not match between the member_enrollment_yearly table and the raw enrollment table, it most likely occurred when cleaning up the plan_type column. Ideally the count difference should be <= 0.

In [26]:
query = '''with truven_enroll as (
    select year, enrolid, plantyp
    from truven.ccaet
    union
    select year, enrolid, plantyp
    from truven.mdcrt
),
truven_plans as (
    select year, d.plan_type, count(*) plan_count
    from truven_enroll m
    left outer join reference_tables.ref_plan_type d
    on d.data_source = 'trv'
  and d.plan_type_src::int = m.plantyp 
    group by 1,2
), dw_plans as (
    select year, plan_type, count(*) plan_count
    from dw_staging.truv_member_enrollment_yearly
    group by 1,2
)
select a.*, b.plan_count as src_plan_count, 
        a.plan_count - b.plan_count as plan_diff, 
        100. * abs(a.plan_count - b.plan_count) / b.plan_count as plan_diff_percent
from truven_plans b
join dw_plans a
on a.year = b.year
and a.plan_type = b.plan_type
order by year;
'''

plan_type_counts_df = pd.read_sql(query,  con=connection)
plan_type_counts_df

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,year,plan_type,plan_count,src_plan_count,plan_diff,plan_diff_percent
0,2011,EPO,1404548,1418205,-13657,0.962978
1,2011,POS,3794460,3856339,-61879,1.604605
2,2011,HMO,6940984,7025611,-84627,1.204550
3,2011,CMP,2561859,2606523,-44664,1.713547
4,2011,PPO,37307217,37497610,-190393,0.507747
...,...,...,...,...,...,...
83,2022,HMO,2574296,2581903,-7607,0.294628
84,2022,CDHP,2199611,2203533,-3922,0.177987
85,2022,EPO,171413,171721,-308,0.179361
86,2022,CMP,588214,592963,-4749,0.800893


In [27]:
plan_type_counts_df.groupby('year')['plan_diff_percent'].min(), plan_type_counts_df.groupby('year')['plan_diff_percent'].max()

(year
 2011    0.507747
 2012    0.406843
 2013    0.535266
 2014    0.658905
 2015    0.476144
 2016    0.410250
 2017    0.288218
 2018    0.315010
 2019    0.354353
 2020    0.422482
 2021    0.500876
 2022    0.177987
 Name: plan_diff_percent, dtype: float64,
 year
 2011    2.501307
 2012    4.064741
 2013    1.543764
 2014    2.586713
 2015    3.029759
 2016    2.940276
 2017    1.316157
 2018    2.136143
 2019    7.304032
 2020    8.132530
 2021    4.150608
 2022    5.573248
 Name: plan_diff_percent, dtype: float64)

In [28]:
plan_type_counts_df.sort_values('plan_diff_percent')

,year,plan_type,plan_count,src_plan_count,plan_diff,plan_diff_percent
84,2022,CDHP,2199611,2203533,-3922,0.177987
85,2022,EPO,171413,171721,-308,0.179361
80,2022,PPO,9626602,9644719,-18117,0.187844
87,2022,POS,2159499,2164525,-5026,0.232199
42,2017,CDHP,3292844,3302362,-9518,0.288218
...,...,...,...,...,...,...
7,2012,EPO,1104730,1151537,-46807,4.064741
79,2021,BMM,3233,3373,-140,4.150608
81,2022,BMM,2965,3140,-175,5.573248
59,2019,BMM,8668,9351,-683,7.304032


In [29]:
plan_types_agg_df = plan_type_counts_df.groupby('year')['plan_count', 'src_plan_count', 'plan_diff'].sum()
plan_types_agg_df

C:\Users\iperez9\AppData\Local\Temp\3\ipykernel_16416\2142812793.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  plan_types_agg_df = plan_type_counts_df.groupby('year')['plan_count', 'src_plan_count', 'plan_diff'].sum()


,plan_count,src_plan_count,plan_diff
year,,,
2011,55894255,56348037,-453782
2012,56032849,56420605,-387756
2013,46917896,47273228,-355332
2014,47019764,47523560,-503796
2015,29624112,29921844,-297732
2016,30207527,30525669,-318142
2017,27044815,27181995,-137180
2018,27411667,27552117,-140450
2019,26487307,26649935,-162628


There are members that do not have a plantyp in the raw data for a given month. Let's see if the number of null plan types decrease at a yearly level

In [30]:
query = '''with truven_enroll as (
    select year, enrolid, plantyp
    from truven.ccaet
    where plantyp is null
    union
    select year, enrolid, plantyp
    from truven.mdcrt
    where plantyp is null
),
truven_plans as (
    select year, count(*) plan_count
    from truven_enroll
    group by 1
), dw_plans as (
    select year, count(*) plan_count
    from dw_staging.truv_member_enrollment_yearly
    where plan_type is null
    group by 1
)
select a.*, b.plan_count as src_plan_count, 
        a.plan_count - b.plan_count as plan_diff, 
        100. * abs(a.plan_count - b.plan_count) / b.plan_count as plan_diff_percent
from truven_plans b
join dw_plans a
on a.year = b.year
order by year;
'''

df = pd.read_sql(query,  con=connection)
df

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,year,plan_count,src_plan_count,plan_diff,plan_diff_percent
0,2011,4567173,4619810,-52637,1.139376
1,2012,4483196,4508459,-25263,0.560347
2,2013,808138,815271,-7133,0.874924
3,2014,3845416,3873721,-28305,0.730693
4,2015,794427,854008,-59581,6.976633
5,2016,518114,533211,-15097,2.831337
6,2017,485795,529393,-43598,8.235470
7,2018,736764,754354,-17590,2.331796
8,2019,455558,503117,-47559,9.452871
9,2020,617484,1120306,-502822,44.882559


## BUS_CD

In [25]:
query = '''with truven_enroll as (
    select year, enrolid, case when medadv = '1' then 'MA' else 'COM' end as bus_cd
    from truven.ccaet
    union
    select year, enrolid, case when medadv = '1' then 'MA' else 'MS' end as bus_cd
    from truven.mdcrt
),
truven_plans as (
    select year, bus_cd, count(*) plan_count
    from truven_enroll
    group by 1,2
), dw_plans as (
    select year, bus_cd, count(*) plan_count
    from dw_staging.truv_member_enrollment_yearly
    group by 1,2
)
select a.*, b.plan_count as src_plan_count, 
        a.plan_count - b.plan_count as plan_diff, 
        100. * abs(a.plan_count - b.plan_count) / b.plan_count as plan_diff_percent
from truven_plans b
full join dw_plans a
on a.year = b.year
and a.bus_cd = b.bus_cd
order by year;
'''

df = pd.read_sql(query,  con=connection)
df

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,year,bus_cd,plan_count,src_plan_count,plan_diff,plan_diff_percent
0,2011,MS,5049811,5243029,-193218,3.685236
1,2011,COM,55411617,55559154,-147537,0.265549
2,2012,MS,4673161,4874717,-201556,4.134722
3,2012,COM,55842884,55975628,-132744,0.237146
4,2013,MS,4093681,4271755,-178074,4.168638
5,2013,COM,43632353,43737217,-104864,0.239759
6,2014,MS,3712364,3868830,-156466,4.044272
7,2014,COM,47152816,47258528,-105712,0.223689
8,2015,MS,2133755,2199633,-65878,2.994954
9,2015,COM,28284784,28348363,-63579,0.224278


In [37]:
df = df.groupby('year')['plan_count', 'src_plan_count', 'plan_diff'].sum()
df['percent_diff'] = 100. * abs(df['plan_count'] - df['src_plan_count']) / df['src_plan_count']
df

C:\Users\iperez9\AppData\Local\Temp\3\ipykernel_16416\4118100362.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df = df.groupby('year')['plan_count', 'src_plan_count', 'plan_diff'].sum()


,plan_count,src_plan_count,plan_diff,percent_diff
year,,,,
2011,60461428,60802183,-340755,0.560432
2012,60516045,60850345,-334300,0.549381
2013,47726034,48008972,-282938,0.589344
2014,50865180,51127358,-262178,0.512794
2015,30418539,30547996,-129457,0.423782
2016,30725641,30856457,-130816,0.423950
2017,27530610,27620062,-89452,0.323866
2018,28148431,28218353,-69922,0.247789
2019,26942865,27026699,-83834,0.310190


## State

In [27]:
query = '''with truven_enroll as (
    select year, enrolid, egeoloc
    from truven.ccaet
    union
    select year, enrolid, egeoloc
    from truven.mdcrt
),
truven_plans as (
    select year::int, case when length(s.abbr) > 2 then '' when m.egeoloc is null then 'null' else s.abbr  end as state, count(*) state_count
    from truven_enroll m
    left outer join reference_tables.ref_truven_state_codes s 
    on m.egeoloc=s.truven_code
    group by 1,2
), dw_plans as (
    select year, case when state is null then 'null' else state end as state, count(*) state_count
    from dw_staging.truv_member_enrollment_yearly
    group by 1,2
)
select a.*, b.state_count as src_state_count, 
        a.state_count - b.state_count as plan_diff, 
        100. * abs(a.state_count - b.state_count) / b.state_count as plan_diff_percent
from truven_plans b
full join dw_plans a
on a.year = b.year
and a.state = b.state
order by year;
'''

df = pd.read_sql(query,  con=connection)
df

,year,state,state_count,src_state_count,plan_diff,plan_diff_percent
0,2011,IL,3466668,3481097,-14429,0.414496
1,2011,NY,4340248,4357569,-17321,0.397492
2,2011,NC,1738609,1752390,-13781,0.786412
3,2011,PR,17561,17782,-221,1.242830
4,2011,AR,296980,300194,-3214,1.070641
...,...,...,...,...,...,...
631,2022,NE,68299,68773,-474,0.689224
632,2022,GA,1191609,1196196,-4587,0.383466
633,2022,DE,101527,102322,-795,0.776959
634,2022,CA,1631181,1639547,-8366,0.510263


We see that there is a large difference in state values assigned using DW logic compared to raw data. Ideally the difference should show that there are more values in the raw data compared to the values in DW due, especially for state values of NULL and ''. When this happens, it means that the member now has a valid state value which can be used to group these member in future projects.

In [24]:
df.sort_values('plan_diff_percent', ascending=False)

,year,state,state_count,src_state_count,plan_diff,plan_diff_percent
184,2014.0,,1185958.0,1631345.0,-445387.0,27.301828
152,2013.0,,1268342.0,1622306.0,-353964.0,21.818572
373,2018.0,PR,875.0,1119.0,-244.0,21.805183
53,2012.0,,1325103.0,1678889.0,-353786.0,21.072626
517,2020.0,PR,975.0,1199.0,-224.0,18.682235
...,...,...,...,...,...,...
285,2016.0,LA,1386764.0,1390084.0,-3320.0,0.238834
546,2021.0,None,10836.0,NaN,NaN,NaN
591,2022.0,None,10428.0,NaN,NaN,NaN
636,NaN,None,NaN,11064.0,NaN,NaN


In [43]:
df['state'].unique()

array(['MA', 'HI', 'MT', 'OH', 'WV', 'WY', 'WA', 'WI', 'AK', 'DE', 'RI',
       'AR', 'IN', 'CA', 'NJ', 'MN', 'OR', 'NC', 'NY', 'NH', 'SD', 'VT',
       'MO', 'TX', 'NV', 'LA', '', 'PR', 'IA', 'ID', 'CO', 'ME', 'GA',
       'MI', 'SC', 'TN', 'MS', 'AL', 'NE', 'CT', 'KS', 'KY', 'NM', 'ND',
       'IL', 'OK', 'MD', 'AZ', 'FL', 'VA', 'UT', 'DC', 'PA'], dtype=object)

In [6]:
df[df['state'] == '']

,year,state,state_count,src_state_count,plan_diff,plan_diff_percent
26,2011,,1772984,1908339.0,-135355.0,7.092817
63,2012,,1325103,1678889.0,-353786.0,21.072626
152,2013,,1268342,1622306.0,-353964.0,21.818572
177,2014,,1185958,1631345.0,-445387.0,27.301828
261,2015,,88356,106195.0,-17839.0,16.798343
302,2016,,95791,116543.0,-20752.0,17.806303
321,2017,,4004163,4068121.0,-63958.0,1.572175
378,2018,,2945112,2987938.0,-42826.0,1.433296
429,2019,,3218770,3264664.0,-45894.0,1.405780
501,2020,,2989527,3031254.0,-41727.0,1.376559


In [7]:
query = '''with truven_enroll as (
    select year, enrolid, egeoloc
    from truven.ccaet
    union
    select year, enrolid, egeoloc
    from truven.mdcrt
)
    select year, case when length(s.abbr) > 2 then '' else s.abbr end as state, count(*) state_count
    from truven_enroll m
    left outer join reference_tables.ref_truven_state_codes s 
    on m.egeoloc=s.truven_code
    group by 1,2'''

temp = pd.read_sql(query,  con=connection)

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [18]:
temp

,year,state,state_count
0,2014.0,MN,410710
1,2021.0,KY,269058
2,2018.0,OK,314323
3,2022.0,NM,56490
4,2019.0,CT,473698
...,...,...,...
631,2018.0,WA,524381
632,2019.0,GA,852054
633,2019.0,LA,264581
634,2021.0,MN,392279


In [15]:
temp[temp['state'].isin([None])]

,year,state,state_count
136,2021.0,None,11064
376,2022.0,None,10524
